In [ ]:
# # General:
# import tweepy           # To consume Twitter's API
# import pandas as pd     # To handle data
# import numpy as np      # For number computing
# import csv
# # For plotting and visualization:
# from IPython.display import display
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
import tweepy
import json
import pandas as pd
from scipy.misc import imread
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib as mpl
import csv
import matplotlib.pyplot as plt
import operator
from textblob import TextBlob
from textblob import Word
from textblob.sentiments import NaiveBayesAnalyzer
# Twitter App access keys for @user

# Consume:
CONSUMER_KEY    = 'YdgIkIh9hUQo3KACKB0Su3Ixy'
CONSUMER_SECRET = 'CqAQf68Zko4CDIefjvitMPX2hxQKsWjnELKOx09TMzrVJjkMi3'

# Access:
ACCESS_TOKEN  = '1233046286-I3HlD2UhO3sGZKWCEZGwn1xCpIW7n2Ljq83minp'
ACCESS_SECRET = 'vm4VdOKIYMk0bh7QpBOuUt04qLrG4Xf8hHSKJyiWzl8D8'

RECENT_TWEETS = 2000

results = []

# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api


# store to dataframe
def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    return data_set

# We create an extractor object:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = tweepy.Cursor(extractor.search,q="#LACvsBAL",count=2000,lang="en",since="2019-01-02").items(RECENT_TWEETS)

# #===================================================
# # Download and save
# # Open/Create a file to append data
# csvFile = open('dataset-2019-01-02.csv', 'a')
# #Use csv Writer
# csvWriter = csv.writer(csvFile)
# # tweepy.Cursor(extractor.search,q="#HappyNewYear2019",count=1000000,lang="en",since="2018-12-31").items()
# for tweet in tweets:
# #     print (tweet.created_at, tweet.text)
#     csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
# #===================================================

# We print the most recent 5 tweets:
print("{0} recent tweets:\n".format(RECENT_TWEETS))
for tweet in tweets:
    results.append(tweet)
    
print("Number of tweets extracted: {}.\n".format(len(results)))

# print(results)
data_set = tweets_df(results)
display(data_set)

# # Remove tweets with duplicate text

text = data_set["text"]

for i in range(0,len(text)):
    txt = ' '.join(word for word in text[i] .split() if not word.startswith('https:'))
    data_set.set_value(i, 'text2', txt)
    
data_set.drop_duplicates('text2', inplace=True)
data_set.reset_index(drop = True, inplace=True)
data_set.drop('text', axis = 1, inplace = True)
data_set.rename(columns={'text2': 'text'}, inplace=True)

text = data_set["text"]

for i in range(0,len(text)):
    textB = TextBlob(text[i])
    sentiment = textB.sentiment.polarity
    data_set.set_value(i, 'Sentiment',sentiment)
    if sentiment <0.00:
        SentimentClass = 'Negative'
        data_set.set_value(i, 'SentimentClass', SentimentClass )
    elif sentiment >0.00:
        SentimentClass = 'Positive'
        data_set.set_value(i, 'SentimentClass', SentimentClass )
    else:
        SentimentClass = 'Neutral'
        data_set.set_value(i, 'SentimentClass', SentimentClass )
data_set.to_csv("TodayAnalysis.csv")

Htag_df = pd.DataFrame()
j = 0

for tweet in range(0,len(results)):
    hashtag = results[tweet].entities.get('hashtags')
    for i in range(0,len(hashtag)):
        Htag = hashtag[i]['text'] 
        Htag_df.set_value(j, 'Hashtag',Htag)
        j = j+1


# Join all the text from the 1000 tweets
Hashtag_Combined = " ".join(Htag_df['Hashtag'].values.astype(str))

no_millennials = " ".join([word for word in Hashtag_Combined.split()
                                if word != 'millennials'
                                and word != 'Millennials'
                                and word != 'Boomers'
                                and word != 'GenX'
                                                                
                                ])

Tweet_mask = imread("./twitter_mask.png", flatten=True)

#Create a Word Cloud
wc = WordCloud(background_color="white", stopwords=STOPWORDS, mask = Tweet_mask)
wc.generate(no_millennials)
plt.imshow(wc)
plt.axis("off")
plt.savefig('./Todays_Hashtag.png', dpi=300)
plt.show()

2000 recent tweets:

